# Encryption, Hashing, and Serialization

In [1]:
from samson.all import *

## Block Ciphers
In samson, block cipher encryption is implemented in a modular and flexible manner. There are block cipher encryption algorithms (e.g. Rijndael, Twofish, RC5, etc) and block cipher modes (e.g. ECB, CBC, GCM, XTS, etc). Block ciphers can only work with bytesrings of an exact size. For example, you can't encrypt 15 or 17 bytes with a 16 byte block cipher. Obviously, the next question is how we encrypt plaintext of arbitrary length with a block cipher. Block cipher modes are the answer. Block cipher modes are algorithms that define a secure (**cough**) way to use the block cipher to encrypt the plaintext. As block cipher modes use an object's functions directly, as long as the functions return `Bytes`, _anything_ can be used as the underlying encryption mechanism.

In [2]:
rij = Rijndael(Bytes.random(32))
cbc = CBC(rij, iv=Bytes.random(rij.block_size))
print(rij)
print(cbc)

<Rijndael: key=<Bytes: b'\x1b]\xec\x08\x9b\xa7\tO\xa8v\xa3\x98\xca\x04\x86\xc6\xe9\xdc\x9a:\xbcyo(\x9fI\x94\xdb\xc8bX=', byteorder=big>, key_size=32, block_size=16>
<CBC: cipher=<Rijndael: key=<Bytes: b'\x1b]\xec\x08\x9b\xa7\tO\xa8v\xa3\x98\xca\x04\x86\xc6\xe9\xdc\x9a:\xbcyo(\x9fI\x94\xdb\xc8bX=', byteorder=big>, key_size=32, block_size=16>, iv=<Bytes: b'\r\xc4\xa6\xb2\xbeK\xe4\x8c\x08\x0fm,Ztt\x16', byteorder=big>>


In [3]:
print(cbc.encrypt(b'my data here'))
print(rij.encrypt(Bytes(b'some other data').zfill(16)))

<Bytes: b'"\xc3\x80\xadq\xf7\x87\x97\x9c?\xc1\xcd[s\x89\xde', byteorder=big>
<Bytes: b'\x88\xb0m\x86\x98\xb3h\x83\xdb\xb8\x00]Rt\t\xa1', byteorder=big>


## Stream Ciphers
Stream ciphers do not have the problem block ciphers have. Stream ciphers are basically CSPRNGs that output arbitrary-long, uniformly-random bytestreams. This bytestream, called the keystream, is then XOR'd with the plaintext. Stream ciphers in samson don't have an `encrypt` function. Instead, they generate the keystream, and it's up to you to XOR. Notice if you generate more keystream, it will be different. Unlike block cipher encryption, stream ciphers have to maintain internal state to prevent constantly generating the same stream.

In [16]:
chacha = ChaCha(key=Bytes.random(32), nonce=Bytes.random(12))
print(chacha)

<ChaCha: key=<Bytes: b'\x04x\x80\xb2W\xfc\x89+\xb0\x1c\x03p\x95\xbe\xe2\xe8\x88\x7f\xf6\xe0\x9f@P\xfb\x13p\xea\xe4TQ\x8c\xf1', byteorder=big>, counter=0, nonce=<Bytes: b'>7C\xefC\x860\xb0\xd5\x05\xce\x8d', byteorder=big>, rounds=20, constant=b'expand 32-byte k'>


In [17]:
plaintext  = b'my plaintext to encrypt'
keystream  = chacha.generate(len(plaintext))
ciphertext = keystream ^ plaintext

print(ciphertext)
print(ciphertext ^ keystream)

<Bytes: b'$\x8b\x86\xed\xa8\xec7\xe8\x01\xf7\x1f\xeb-\xfd\tR\xc7^:M\x01\xb3\x03', byteorder=little>
<Bytes: b'my plaintext to encrypt', byteorder=little>


## Asymmetric Encryption
Asymmetric or public key encryption uses two keys to perform encryption and decryption. These keys are mathematically related and are usually built from number theoretical constructions. As public key cryptography requires dissemination of the public key, several encoding and serialization standards have been devised to compactly represent them. samson's public key encoding includes all of the major schemes such as PKCS1, PKCS8, JWK, X509, and even OpenSSH. Some of these required reverse engineering of custom formats and algorithms (OpenSSH) or out-of-spec encodings (OpenSSL's DSA/ECDSA PKCS1 encoding).

Note that every `PKIEncoding` takes in `**kwargs` and processes parameters specific to them.

In [27]:
rsa = RSA(1024)
print(rsa)
print()

rsa.encrypt(b'hiya')

print(rsa.export_private_key(encoding=PKIEncoding.OpenSSH).decode())
print()
print(rsa.export_public_key(encoding=PKIEncoding.X509).decode())
print()

# Creating a CA cert
cn = 'CN=hiya,O=hiya-corp,L=Rack City'
print(rsa.export_public_key(encoding=PKIEncoding.X509_CERT, ca=True, serial_number=666, subject=cn, issuer=cn).decode())

new_rsa = RSA.import_key(rsa.export_private_key(encoding=PKIEncoding.PKCS1))

<RSA: bits=1024, p=12874646079746165555911313540195212672424385516548593813129400478799954469843535037146623170967209338176146858661595005442057507236141669522637607430635613, q=12147042868817769450574951420187935779650349373986353564858872804086153156462279627420576712526311589884889641028026468251277825968849649092766677070141921, e=65537, n=156388877851533311817260750373173726317504553196108017557455609904996721427290923450197906726841262899324865464891956291789862820790813255582515848823284510335409617679143587998003812685973372592735048563007512491067816612565889100029652899165599991504274222000540711971967448414474946985365587896657146832573, phi=39097219462883327954315187593293431579376138299027004389363902476249180356822730862549476681710315724831216366222989072947465705197703313895628962205821121328430167278802145377934713075706230129500039507015033625698633431614565698553747082991429124495836540241010255587623438769785435498511743123093161513760, d=283727323138796569496197616

In [30]:
ecdsa = ECDSA(P256.G, hash_obj=SHA512())
print(ecdsa)
print()

message = b'data to sign'
r,s     = ecdsa.sign(message)
print('Signature valid?', ecdsa.verify(message, (r,s)))
print()

print(ecdsa.export_public_key(encoding=PKIEncoding.SSH2).decode())

<ECDSA: d=79911338412996220638515983283727336924764927230192709548511111270057951212169, G=WeierstrassCurve{a=-3, b=41058363725152142129326129780047268409114441015993725554835256314039467401291}({x=ZZ(48439561293906451759052585252797914202762949526041747995844080717082404635286), y=ZZ(36134250956749795798585127919587881956611106672985015071877198253568414405109)}), Q=WeierstrassCurve{a=-3, b=41058363725152142129326129780047268409114441015993725554835256314039467401291}({x=ZZ(68142404019967274175624892604822204756235887636304889337874750548409119726776), y=ZZ(37123609755519743280863973255747440733775007684661087322340254284018791564622)}), hash_obj=<SHA2: initial_state=<Bytes: b'j\t\xe6g\xf3\xbc\xc9\x08\xbbg\xae\x85\x84\xca\xa7;<n\xf3r\xfe\x94\xf8+\xa5O\xf5:_\x1d6\xf1Q\x0eR\x7f\xad\xe6\x82\xd1\x9b\x05h\x8c+>l\x1f\x1f\x83\xd9\xab\xfbA\xbdk[\xe0\xcd\x19\x13~!y', byteorder=big>, block_size=128, digest_size=64>>

Signature valid? True

---- BEGIN SSH2 PUBLIC KEY ----
AAAAE2VjZHNhLXNoYTItbml

## Hashing Algorithms
Hashing algorithms take in an arbitrary number of bytes and return a fixed-size, uniformly-random hash. The algorithms can be parameterized for your needs and allow full introspection. A common parameter is `initial_state`. It defines the starting point of the hash and, in most other libraries, is fixed. However, samson allows you to change internals on a whim. This is useful for executing attacks like the length-extension attack.

In [3]:
print(MD5().hash(b''))
print(SHA1().hash(b''))
print(SHA256().hash(b''))
print(SHA512().hash(b''))
print(Whirlpool().hash(b''))
print(BLAKE2b().hash(b''))

print()
print(MD5(initial_state=Bytes.random(16)).hash(b''))

<Bytes: b'\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x98\xec\xf8B~', byteorder=big>
<Bytes: b'\xda9\xa3\xee^kK\r2U\xbf\xef\x95`\x18\x90\xaf\xd8\x07\t', byteorder=big>
<Bytes: b"\xe3\xb0\xc4B\x98\xfc\x1c\x14\x9a\xfb\xf4\xc8\x99o\xb9$'\xaeA\xe4d\x9b\x93L\xa4\x95\x99\x1bxR\xb8U", byteorder=big>
<Bytes: b"\xcf\x83\xe15~\xef\xb8\xbd\xf1T(P\xd6m\x80\x07\xd6 \xe4\x05\x0bW\x15\xdc\x83\xf4\xa9!\xd3l\xe9\xceG\xd0\xd1<]\x85\xf2\xb0\xff\x83\x18\xd2\x87~\xec/c\xb91\xbdGAz\x81\xa582z\xf9'\xda>", byteorder=big>
<Bytes: b'\x19\xfaa\xd7U"\xa4f\x9bD\xe3\x9c\x1d.\x17&\xc50#!0\xd4\x07\xf8\x9a\xfe\xe0\x96I\x97\xf7\xa7>\x83\xbei\x8b(\x8f\xeb\xcf\x88\xe3\xe0<O\x07W\xea\x89d\xe5\x9bc\xd97\x08\xb18\xccB\xa6n\xb3', byteorder=big>
<Bytes: b'xj\x02\xf7B\x01Y\x03\xc6\xc6\xfd\x85%R\xd2r\x91/G@\xe1XGa\x8a\x86\xe2\x17\xf7\x1fT\x19\xd2^\x101\xaf\xeeXS\x13\x89dD\x93N\xb0K\x90:h[\x14H\xb7U\xd5op\x1a\xfe\x9b\xe2\xce', byteorder=little>

<Bytes: b'\xd9\x07a:\x19\xf8\x14\xab\xfd\xd6\x07\x0epH\xe5\xdc', byteorder=big>
